In [1]:
import pandas as pd
import numpy as np

In [2]:
# load the starts data
starts_df = pd.read_csv('../capstone_data/skillshare_2022_starts.csv')

/Users/travisstowe/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (24,53) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [7]:
# clean the starts data.
starts_df = starts_df[starts_df['plan_length'] == 12]
starts_df = starts_df[starts_df['is_team'] == False]
starts_df = starts_df[starts_df['is_scholarship'] == False]
starts_df = starts_df[starts_df['is_direct_to_paid'] == False]
starts_df = starts_df[starts_df['had_trial'] == True]
starts_df = starts_df[starts_df['trial_length_offer'].isin(['One Week', 'One Month'])]

In [11]:
# all of these columns have the chance to be valuable to us.
valuable_cols = ['user_uid', 'is_active', 'create_time',
       'first_payment_time', 'last_payment_time', 'next_billing_time',
       'last_payment_attempt', 'last_failed_payment_attempt',
       'user_cancellation_time', 'cancellation_time', 'refund_time', 'coupon_id',
       'coupon_trial_length', 'payment_provider', 'payment_ux', 'is_paused',
       'pause_time', 'is_refunded', 'is_resume', 'paid_through', 'is_lapsed', 'is_pending', 'is_cancelled',
       'has_paid', 'is_paid', 'trial_end', 'category',
       'num_payments', 'num_successful_payments',
       'first_payment_currency_code', 'subscription_number',
       'paid_subscription_number', 'eligible_subscription_number',
       'is_p1_refunded', 'trial_extension_time',
       'original_create_time', 'original_trial_end', 'extended_trial_end',
       'was_trial_extended', 'is_trial_extension', 'is_split_trial',
       'is_paid_reactivation', 'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign',
       'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source',
       'eligible_trial_number']

starts_big_clean_df = starts_df[valuable_cols]

In [12]:
# these are the columns I bet will have some value to prediction.
prediction_cols = ['user_uid', 'create_time', 'first_payment_time', 'last_payment_attempt',
                   'last_failed_payment_attempt', 'user_cancellation_time', 'cancellation_time', 
                   'refund_time', 'coupon_id', 'coupon_trial_length', 'payment_provider', 'payment_ux', 
                   'is_refunded', 'has_paid', 'trial_end', 'first_payment_currency_code', 'original_trial_end', 
                   'extended_trial_end', 'was_trial_extended', 'is_trial_extension', 'is_split_trial', 
                   'trial_length_days', 'trial_length_offer', 'sub_utm_source', 'sub_utm_campaign', 
                   'sub_utm_medium', 'sub_utm_term', 'sub_utm_channel', 'referral_source', 'eligible_trial_number']

starts_clean_df = starts_big_clean_df[prediction_cols]

In [39]:
# make a day version of the trial start and end dates.
starts_clean_df['trial_end_day'] = pd.to_datetime(starts_clean_df.original_trial_end).dt.date
starts_clean_df['trial_start_day'] = pd.to_datetime(starts_clean_df.create_time).dt.date

# for some reason user_uid is a float only on this table. Change it to an int.
starts_clean_df['user_uid'] = starts_clean_df['user_uid'].astype(int)

<ipython-input-39-15df73d7d27c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df['trial_end_day'] = pd.to_datetime(starts_clean_df.original_trial_end).dt.date
<ipython-input-39-15df73d7d27c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  starts_clean_df['trial_start_day'] = pd.to_datetime(starts_clean_df.create_time).dt.date
<ipython-input-39-15df73d7d27c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [41]:
# make a small df for merging onto the video views data.
trial_ends = starts_clean_df[['user_uid', 'trial_start_day', 'trial_end_day']]

In [42]:
# load the video views data.
vviews1_df = pd.read_csv('../capstone_data/skillshare_2022_vviews_0.csv')

In [ ]:
# change str view_date into dt.
vviews1_df['view_date_dt'] = pd.to_datetime(vviews1_df.view_date).dt.date

In [43]:
# merge the trial start and end dates onto video views.
vviews1_df = vviews1_df.merge(trial_ends, how='left', left_on='uid', right_on='user_uid')

In [46]:
# filter to views that happened during the trial
trial_vviews1_df = vviews1_df[vviews1_df['view_date_dt'] < vviews1_df['trial_end_day']]

In [48]:
# add a column that tells what day of the trial the view occured, 
# thus normalizing everyone to the same "start date" of "1"
trial_vviews1_df['day_of_trial'] = (trial_vviews1_df['view_date_dt'] - trial_vviews1_df['trial_start_day']).dt.days + 1

<ipython-input-48-08956d5b25d4>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trial_vviews1_df['day_of_trial'] = (trial_vviews1_df['view_date_dt'] - trial_vviews1_df['trial_start_day']).dt.days + 1


In [57]:
# some folks have had multiple trials, so they can have video views from dates prior to the trial dates here.
# preferrably we would split that into two different trials and analyze
# this isn't perfect, but for now, I'm just filtering out the previous trials and only considering
# views from the trial that fit the date range.
trial_vviews1_df = trial_vviews1_df[trial_vviews1_df['day_of_trial'] > 0]

In [58]:
# group by minutes watched by day.  Not super sophisticated.  Tons could be done here.
ttv_df = trial_vviews1_df.groupby(by=['uid', 'day_of_trial']).agg(minutes_watched=pd.NamedAgg(column='sum', aggfunc='sum')).reset_index()

In [68]:
# pivot the data and fill nas with 0
vviews_pivot = ttv_df.pivot_table(
    index=['uid'],
    columns='day_of_trial',
    values='minutes_watched').reset_index().rename_axis(None, axis=1).reset_index(drop=True).fillna(0.0)

In [69]:
vviews_pivot

,uid,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,526,2229.0,65.0,0.0,1177.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,110634,840.0,925.0,0.0,1173.0,0.0,1197.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,119298,895.0,51.0,0.0,719.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,247585,621.0,0.0,3259.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,297525,4070.0,0.0,0.0,2748.0,3077.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,864.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5142,23789787,2124.0,2216.0,0.0,1185.0,0.0,0.0,2881.0,565.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5143,23789860,4353.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1062.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5144,23789963,2742.0,1481.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5145,23790057,1469.0,0.0,2236.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
